In [1]:
import os
import numpy as np
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import  os
import pickle
from pathlib import Path


out_path = "/home/mymusise/Downloads/pangu_model/numpy_weights"

path = Path(out_path)

files = path.glob('*.pickle')
files = sorted(list(files), key=lambda x:x.stat().st_mtime)

weights = {}

for f in files:
    f = str(f)
    key = f.split('/')[-1]
    keys = key.split('.')[:-1]
    key = ".".join(keys)
    print(key)
    with open(f, 'rb') as f:
        w_list = pickle.load(f)
        if type(w_list[0]) == np.ndarray and w_list[0].shape:                
            w_list = np.concatenate(w_list, axis=0)
        weights[key] = w_list

backbone.blocks.0.layernorm1.gamma
backbone.blocks.0.layernorm1.beta
backbone.blocks.0.layernorm2.gamma
backbone.blocks.0.layernorm2.beta
backbone.blocks.0.attention.projection.weight
backbone.blocks.0.attention.projection.bias
backbone.blocks.0.attention.dense1.weight
backbone.blocks.0.attention.dense1.bias
backbone.blocks.0.attention.dense2.weight
backbone.blocks.0.attention.dense2.bias
backbone.blocks.0.attention.dense3.weight
backbone.blocks.0.attention.dense3.bias
backbone.blocks.0.output.mapping.weight
backbone.blocks.0.output.mapping.bias
backbone.blocks.0.output.projection.weight
backbone.blocks.0.output.projection.bias
backbone.blocks.1.layernorm1.gamma
backbone.blocks.1.layernorm1.beta
backbone.blocks.1.layernorm2.gamma
backbone.blocks.1.layernorm2.beta
backbone.blocks.1.attention.projection.weight
backbone.blocks.1.attention.projection.bias
backbone.blocks.1.attention.dense1.weight
backbone.blocks.1.attention.dense1.bias
backbone.blocks.1.attention.dense2.weight
backbone.blo

In [3]:
def get_weight(name):
    for key in weights.keys():
        if name in key:
            return weights[key]

# print(get_weight(f'{block_name}.layernorm1.gamma'))

In [4]:
for k, v in weights.items():
    if hasattr(v, 'shape'):
        print(k, v.shape)

backbone.blocks.0.layernorm1.gamma (1310720,)
backbone.blocks.0.layernorm1.beta (1310720,)
backbone.blocks.0.layernorm2.gamma (1310720,)
backbone.blocks.0.layernorm2.beta (1310720,)
backbone.blocks.0.attention.projection.weight (2560, 2560)
backbone.blocks.0.attention.projection.bias (1310720,)
backbone.blocks.0.attention.dense1.weight (2560, 2560)
backbone.blocks.0.attention.dense1.bias (2560,)
backbone.blocks.0.attention.dense2.weight (2560, 2560)
backbone.blocks.0.attention.dense2.bias (2560,)
backbone.blocks.0.attention.dense3.weight (2560, 2560)
backbone.blocks.0.attention.dense3.bias (2560,)
backbone.blocks.0.output.mapping.weight (20480, 1280)
backbone.blocks.0.output.mapping.bias (10240,)
backbone.blocks.0.output.projection.weight (10240, 2560)
backbone.blocks.0.output.projection.bias (1310720,)
backbone.blocks.1.layernorm1.gamma (1310720,)
backbone.blocks.1.layernorm1.beta (1310720,)
backbone.blocks.1.layernorm2.gamma (1310720,)
backbone.blocks.1.layernorm2.beta (1310720,)
bac

In [5]:
from models import TFPanGuAlphaLMHeadModel
from transformers import GPT2Config

config = GPT2Config(
    vocab_size=40000,
    n_positions=1024,
    n_ctx=1024,
    n_embd=2560,
    n_layer=31,
    n_head=32
)
model = TFPanGuAlphaLMHeadModel(config)
input = tf.constant([[1, 2]])
out = model(input)[0]
print(out.shape)

(1, 2, 40000)


In [6]:
import re
new_weights = []

download_path = '/home/mymusise/Downloads/pangu_model/'
not_set = []
for weight in model.weights:
    num_layer = 0
    if 'h_._' in weight.name:
        num_layer = re.findall(r'transformer/h_._(\d+)', weight.name)[0]
        block_name = f"backbone.blocks.{num_layer}"
    if 'top_query_layer' in weight.name:
        block_name = 'backbone.top_query_layer'
    
    if 'transformer/wte/weight:0' in weight.name:
        w = np.load(f"{download_path}/word_embedding.npy")
        assert w.shape == (40000, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting transformer/wte/weight')
    elif 'transformer/wpe/embeddings:0' in weight.name:
        w = np.load(f"{download_path}/position_embedding.npy")
        assert w.shape == (1024, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting transformer/wpe/embeddings')
    elif 'top_query_embedding' in weight.name:
        w = np.load(f"{download_path}/top_query_embedding.npy")
        assert w.shape == (1024, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting transformer/top_query_embedding')
    elif f'/ln_1/gamma:0' in weight.name:
        w = get_weight(f'{block_name}.layernorm1.gamma')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == (2560, )
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif f'/ln_1/beta:0' in weight.name:
        w = get_weight(f'{block_name}.layernorm1.beta')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == (2560, )
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')


    ### normal transformer  ###
    elif 'attn/c_attn/weight:0' in weight.name and 'top_query' not in weight.name:
        query = get_weight(f'{block_name}.attention.dense1.weight')
        key = get_weight(f'{block_name}.attention.dense2.weight')
        value = get_weight(f'{block_name}.attention.dense3.weight')
        w = np.concatenate([query, key, value], axis=1)
        assert w.shape == (2560, 7680)
        # w = np.transpose(w)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_attn/bias:0' in weight.name and 'top_query' not in weight.name:
        query = get_weight(f'{block_name}.attention.dense1.bias')
        key = get_weight(f'{block_name}.attention.dense2.bias')
        value = get_weight(f'{block_name}.attention.dense3.bias')
        w = np.concatenate([query, key, value], axis=0)
        w = w.reshape(1,7680) # ???
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_proj/weight:0' in weight.name and 'top_query' not in weight.name:
        w = get_weight(f'{block_name}.attention.projection.weight')
        assert w.shape == (2560, 2560)
        # w = np.transpose(w)  # ???
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_proj/bias:0' in weight.name and 'top_query' not in weight.name:
        w = get_weight(f'{block_name}.attention.projection.bias')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        w = w.reshape(1, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')


    ###  top_quert ###
    elif 'attn/c_attn/weight:0' in weight.name and 'top_query' in weight.name:
        # query = get_weight(f'{block_name}.attention.dense1.weight')
        key = get_weight(f'{block_name}.attention.dense2.weight')
        value = get_weight(f'{block_name}.attention.dense3.weight')
        w = np.concatenate([key, value], axis=1)
        assert w.shape == (2560, 2560 * 2)
        print(w.shape, weight.shape)
        # w = np.transpose(w)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_attn/bias:0' in weight.name and 'top_query' in weight.name:
        # query = get_weight(f'{block_name}.attention.dense1.bias')
        key = get_weight(f'{block_name}.attention.dense2.bias')
        value = get_weight(f'{block_name}.attention.dense3.bias')
        w = np.concatenate([key, value], axis=0)
        w = w.reshape(1,2560 * 2)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_attn_query/weight:0' in weight.name and 'top_query' in weight.name:
        w = get_weight(f'{block_name}.attention.dense1.weight')
        assert w.shape == (2560, 2560)
        # w = np.transpose(w)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_attn_query/bias:0' in weight.name and 'top_query' in weight.name:
        w = get_weight(f'{block_name}.attention.dense1.bias')
        w = w.reshape(1,2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_proj/weight:0' in weight.name and 'top_query' in weight.name:
        w = get_weight(f'{block_name}.attention.projection.weight')
        assert w.shape == (2560, 2560)
        # w = np.transpose(w)  # ???
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'attn/c_proj/bias:0' in weight.name and 'top_query' in weight.name:
        w = get_weight(f'{block_name}.attention.projection.bias')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        w = w.reshape(1, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    ### top query ###


    elif f'/ln_2/gamma:0' in weight.name:
        w = get_weight(f'{block_name}.layernorm2.gamma')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == (2560, )
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif f'/ln_2/beta:0' in weight.name:
        w = get_weight(f'{block_name}.layernorm2.beta')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == (2560, )
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'mlp/c_fc/weight:0' in weight.name:
        w = get_weight(f'{block_name}.output.mapping.weight')
        assert w.shape == (10240 * 2, 2560 /2)  # TODO: 这里很神奇，要检查下
        w = np.reshape(w, (2560, 10240))
        # assert w.shape == (10240, 2560)
        # w = np.transpose(w)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'mlp/c_fc/bias:0' in weight.name:
        w = get_weight(f'{block_name}.output.mapping.bias')
        assert w.shape == (10240, )
        w = w.reshape(1, 10240)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'mlp/c_proj/weight:0' in weight.name:
        w = get_weight(f'{block_name}.output.projection.weight')
        assert w.shape == (10240, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'mlp/c_proj/bias:0' in weight.name:
        w = get_weight(f'{block_name}.output.projection.bias')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        w = w.reshape(1, 2560)
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting layer {weight.name}')
    elif 'transformer/ln_f/gamma:0' in weight.name:
        w = get_weight(f'backbone.layernorm.gamma')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting transformer/ln_f/gamma')
    elif 'transformer/ln_f/beta:0' in weight.name:
        w = get_weight(f'backbone.layernorm.beta')
        assert w.shape == (2560 * 512, )
        w = w[:2560,]
        assert w.shape == weight.shape
        new_weights.append(w)
        print(f'setting transformer/ln_f/beta')
    else:
        not_set.append(weight.name)

print(not_set)
print(len(model.weights), len(new_weights))
model.set_weights(new_weights)

setting transformer/wpe/embeddings
setting transformer/wte/weight
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/ln_1/gamma:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/ln_1/beta:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/attn/c_attn/weight:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/attn/c_attn/bias:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/attn/c_proj/weight:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/attn/c_proj/bias:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/ln_2/gamma:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/ln_2/beta:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/mlp/c_fc/weight:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/mlp/c_fc/bias:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/mlp/c_proj/weight:0
setting layer tf_pan_gu_alpha_lm_head_model/transformer/h_._0/mlp/c_

In [8]:
from transformers import XLNetTokenizer, TFGPT2LMHeadModel
# add spicel process 
class XLNetTokenizer(XLNetTokenizer):
    translator = str.maketrans(" \n", "\u2582\u2583")

    def _tokenize(self, text, *args, **kwargs):
        text = [x.translate(self.translator) for x in jieba.cut(text, cut_all=False)]
        text = " ".join(text)
        return super()._tokenize(text, *args, **kwargs)

    def _decode(self, *args, **kwargs):
        text = super()._decode(*args, **kwargs)
        text = text.replace(' ', '').replace('\u2582', ' ').replace('\u2583', '\n')
        return text


tokenizer = XLNetTokenizer.from_pretrained('/data2/PanGu-Alpha')


from transformers import TextGenerationPipeline
import jieba


# class TextGenerationPipeline(TextGenerationPipeline):

TextGenerationPipeline.ALLOWED_MODELS.append("TFPanGuAlphaLMHeadModel")


text_generater = TextGenerationPipeline(model, tokenizer)

texts = [
    '我的理想',
    # '天下武功, 唯快不',
    # '天下熙熙,',
    # '她腰间',
    # """
    # 我们在火星上发现了大量的神奇物种。有神奇的海星兽，身上是粉色的，有5条腿；有胆小的猫猫兽，橘色，有4条腿；有令人恐惧的蜈蚣兽，全身漆黑，36条腿；有纯洁的天使兽，全身洁白无瑕，有3条腿；有贪吃的汪汪兽，银色的毛发，有5条腿；有蛋蛋兽，紫色，8条腿。

    # 请根据上文，列出一个表格，包含物种名、颜色、腿数量。
    # |物种名|颜色|腿数量|
    # |亚古兽|金黄|2|
    # |海星兽|粉色|5|
    # |猫猫兽|橘色|4|
    # |蜈蚣兽|漆黑|36|
    # """
]

for text in texts:
    token_len = len(tokenizer._tokenize(text))
    # print(text_generater(text, max_length=token_len + 200, top_k=1, use_cache=True, prefix='')[0]['generated_text'])
    print(text_generater(text, max_length=token_len + 16, do_sample=True, repetition_penalty=1.3)[0]['generated_text'])
    print(text_generater(text, max_length=token_len + 16, do_sample=True, top_k=3)[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
我的理想三四,,双特别<eot>就下2“,和上
我的理想,的,,,,,的的
